In [1]:
import pandas as pd
from sklearn.ensemble import ExtraTreesRegressor
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from decimal import Decimal
from joblib import parallel_backend
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df1 = pd.read_csv('train_set/context--2008-08-22_2010-07-10--saaf.csv')
df1["convertedDate"] = pd.to_datetime(df1["ut_ms"],unit="ms") 

In [3]:
df2 = pd.read_csv('train_set/context--2010-07-10_2012-05-27--saaf.csv')
df2["convertedDate"] = pd.to_datetime(df2["ut_ms"],unit="ms") 

In [4]:
df3 = pd.read_csv('train_set/context--2012-05-27_2014-04-14--saaf.csv')
df3["convertedDate"] = pd.to_datetime(df3["ut_ms"],unit="ms") 

In [5]:
df_inter = pd.concat([df1, df2])
df = pd.concat([df_inter , df3])
df

,ut_ms,sa,sx,sy,sz,convertedDate
0,1219363213000,0.32,14.55,90.32,104.55,2008-08-22 00:00:13
1,1219363235000,0.34,14.56,90.34,104.55,2008-08-22 00:00:35
2,1219363295000,0.34,14.56,90.34,104.55,2008-08-22 00:01:35
3,1219363355000,0.34,14.56,90.34,104.55,2008-08-22 00:02:35
4,1219363415000,0.34,14.56,90.34,104.55,2008-08-22 00:03:35
...,...,...,...,...,...,...
755071,1397433344000,1.41,1.57,88.59,89.32,2014-04-13 23:55:44
755072,1397433404000,1.41,1.57,88.59,89.32,2014-04-13 23:56:44
755073,1397433464000,1.41,1.57,88.59,89.32,2014-04-13 23:57:44
755074,1397433524000,1.41,1.57,88.59,89.32,2014-04-13 23:58:44


In [6]:
df['timeDiff'] = (df['ut_ms'].diff())/1000

In [7]:
df= df.drop_duplicates()
df

,ut_ms,sa,sx,sy,sz,convertedDate,timeDiff
0,1219363213000,0.32,14.55,90.32,104.55,2008-08-22 00:00:13,NaN
1,1219363235000,0.34,14.56,90.34,104.55,2008-08-22 00:00:35,22.0
2,1219363295000,0.34,14.56,90.34,104.55,2008-08-22 00:01:35,60.0
3,1219363355000,0.34,14.56,90.34,104.55,2008-08-22 00:02:35,60.0
4,1219363415000,0.34,14.56,90.34,104.55,2008-08-22 00:03:35,60.0
...,...,...,...,...,...,...,...
755071,1397433344000,1.41,1.57,88.59,89.32,2014-04-13 23:55:44,60.0
755072,1397433404000,1.41,1.57,88.59,89.32,2014-04-13 23:56:44,60.0
755073,1397433464000,1.41,1.57,88.59,89.32,2014-04-13 23:57:44,60.0
755074,1397433524000,1.41,1.57,88.59,89.32,2014-04-13 23:58:44,60.0


In [8]:
df.isnull().sum()

ut_ms            0
sa               0
sx               0
sy               0
sz               0
convertedDate    0
timeDiff         1
dtype: int64

In [9]:
resampled = df.resample("1H", on='convertedDate').mean()
resampled

,ut_ms,sa,sx,sy,sz,timeDiff
convertedDate,,,,,,
2008-08-22 00:00:00,1.219365e+12,0.339355,14.551129,90.339355,104.548871,58.393443
2008-08-22 01:00:00,1.219369e+12,6.619310,18.012184,90.500460,90.396437,41.471264
2008-08-22 02:00:00,1.219372e+12,30.927137,37.511806,64.618150,86.910793,15.863436
2008-08-22 03:00:00,1.219376e+12,1.534306,15.217917,89.967083,105.197917,49.875000
2008-08-22 04:00:00,1.219379e+12,0.339672,14.523934,90.339672,104.520492,59.016393
...,...,...,...,...,...,...
2014-04-13 19:00:00,1.397417e+12,0.511176,35.487500,89.832941,125.196912,52.941176
2014-04-13 20:00:00,1.397421e+12,2.033043,21.455326,89.704130,110.950543,39.130435
2014-04-13 21:00:00,1.397425e+12,0.936024,0.484940,89.614819,89.847349,43.373494


In [10]:
resampled.isnull().sum()

ut_ms       1846
sa          1846
sx          1846
sy          1846
sz          1846
timeDiff    1846
dtype: int64

In [11]:
resampled = resampled.ffill().bfill()

In [12]:
resampled.isnull().sum()

ut_ms       0
sa          0
sx          0
sy          0
sz          0
timeDiff    0
dtype: int64

In [13]:
df_test = pd.read_csv('test_set/context--2014-04-14_2016-03-01--saaf.csv')
df_test["convertedDate"] = pd.to_datetime(df_test["ut_ms"],unit="ms") 
df_test.isnull().sum()

ut_ms            0
sa               0
sx               0
sy               0
sz               0
convertedDate    0
dtype: int64

In [14]:
resampled_test = df_test.resample("1H", on='convertedDate').mean()
resampled_test

,ut_ms,sa,sx,sy,sz
convertedDate,,,,,
2014-04-14 00:00:00,1.397435e+12,16.619691,35.934330,104.483918,117.566804
2014-04-14 01:00:00,1.397439e+12,30.485949,69.282532,119.638861,141.674304
2014-04-14 02:00:00,1.397442e+12,10.349634,5.559756,90.604878,93.674268
2014-04-14 03:00:00,1.397446e+12,1.410714,1.513714,88.589286,89.446714
2014-04-14 04:00:00,1.397450e+12,1.411429,1.501143,88.588571,89.481714
...,...,...,...,...,...
2016-02-29 19:00:00,1.456774e+12,1.141722,12.854000,90.070778,102.853889
2016-02-29 20:00:00,1.456778e+12,0.634278,33.522111,90.152222,123.522056
2016-02-29 21:00:00,1.456781e+12,2.344410,32.912050,90.093975,122.912050


In [15]:
resampled_test.isnull().sum()

ut_ms    566
sa       566
sx       566
sy       566
sz       566
dtype: int64

In [16]:
resampled_test = resampled.ffill().bfill()
resampled_test.isnull().sum()

ut_ms       0
sa          0
sx          0
sy          0
sz          0
timeDiff    0
dtype: int64

In [17]:
resampled = resampled.drop(['timeDiff'], axis=1)

In [18]:
resampled = resampled.reset_index()
resampled_test = resampled_test.reset_index()

In [19]:
resampled  = resampled.rename(columns={"convertedDate": "timestamp"})
resampled_test  = resampled.rename(columns={"convertedDate": "timestamp"})
resampled = resampled.drop(['ut_ms'],axis=1)

In [20]:
resampled.to_csv("saafTrainF.csv",index=False)
resampled_test.to_csv("saafTestF.csv",index=False)

In [21]:
ltdata_train = pd.read_csv('ltdata_train.csv')
dmop_train = pd.read_csv('dmop_train.csv')
evtf_train = pd.read_csv('evtf_train.csv')
ftl_train = pd.read_csv('ftl_train.csv')
power_train  = pd.read_csv('power_train.csv')

In [22]:
ltdata_train  = ltdata_train.rename(columns={"ut_ms": "timestamp"})
dmop_train  = dmop_train.rename(columns={"ut_ms": "timestamp"})
ftl_train  = ftl_train.rename(columns={"ut_ms": "timestamp"})

In [23]:
power_train['timestamp'] = pd.to_datetime(power_train['timestamp'])
power_train

,timestamp,NPWD2372,NPWD2401,NPWD2402,NPWD2451,NPWD2471,NPWD2472,NPWD2481,NPWD2482,NPWD2491,...,NPWD2792,NPWD2801,NPWD2802,NPWD2821,NPWD2851,NPWD2852,NPWD2871,NPWD2872,NPWD2881,NPWD2882
0,2008-08-22 00:00:00,0.001821,0.001474,0.172173,2.175650,0.000714,0.000626,0.000554,0.001827,0.424989,...,0.000443,0.001474,0.156296,0.004438,0.004982,0.000501,0.000499,0.000714,0.006457,0.003873
1,2008-08-22 01:00:00,0.001821,0.001474,0.172173,2.175650,0.000714,0.000626,0.000554,0.001827,0.424989,...,0.000443,0.001474,0.156296,0.004438,0.004982,0.000501,0.000499,0.000714,0.006457,0.003873
2,2008-08-22 02:00:00,0.368583,0.001474,0.177440,0.005015,0.000714,0.000626,0.000554,0.001827,0.002016,...,0.000553,0.001474,0.002389,0.005547,0.004982,0.000602,0.000499,0.000714,0.006457,0.003227
3,2008-08-22 03:00:00,0.001821,0.001474,0.177440,0.006019,0.000714,0.000751,0.000554,0.001827,0.435523,...,0.000553,0.001474,0.325832,0.005547,0.004982,0.000501,0.000598,0.000714,0.007748,0.003873
4,2008-08-22 04:00:00,0.001821,0.001474,0.172173,2.175650,0.000714,0.000626,0.000554,0.001827,0.002016,...,0.000553,0.001474,0.002123,0.005547,0.004982,0.000501,0.000499,0.000857,0.006457,0.003227
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49459,2014-04-13 19:00:00,0.373872,0.001474,0.172173,2.175650,0.000714,0.006247,0.000665,0.001827,0.002016,...,0.000996,0.001769,0.002389,0.006657,0.408604,0.000501,0.000598,0.000857,0.006457,0.003227
49460,2014-04-13 20:00:00,0.373872,0.001474,0.172173,0.005015,0.000714,0.000876,0.000554,0.001827,0.002016,...,0.000996,0.001474,0.002123,0.005547,0.886972,0.000501,0.000499,0.000714,0.006457,0.003227
49461,2014-04-13 21:00:00,0.001821,0.001474,0.172173,0.005015,0.000714,0.001001,0.000554,0.001827,0.002016,...,0.000996,0.001474,0.156296,0.005547,0.388672,0.000501,0.000499,0.000714,0.006457,0.003227
49462,2014-04-13 22:00:00,0.001821,0.001769,0.177440,1.112890,0.000714,0.001001,0.000665,0.002192,0.002016,...,0.000996,0.001474,0.325832,0.005547,0.069760,0.000602,0.000499,0.000857,0.006457,0.003227


In [24]:
T1=pd.merge(ltdata_train , dmop_train, on=["timestamp"])
T2 = pd.merge(T1 , evtf_train, on=["timestamp"])
T3 = pd.merge(T2 , ftl_train, on=["timestamp"])
T3['timestamp'] = pd.to_datetime(T3['timestamp'])
train = pd.merge(T3 , resampled, on=["timestamp"])
mergedTrain = pd.merge(train , power_train, on=["timestamp"])